# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import geoviews
import pyproj
import panel as pn
import warnings
import sys
import os

# Add the directory containing api_keys.py to Python's path
sys.path.append(r'C:\Users\lunar01\python-api-challenge\WeatherPy')

# Now import the API key
from api_keys import geoapify_key

# Suppress warnings
warnings.filterwarnings("ignore")

# Print Panel version
print("Panel version:", pn.__version__)

# Import panel.depends and print its directory
import panel.depends as depends
print("\nPanel.depends directory contents:")
print(dir(depends))



Panel version: 1.5.4

Panel.depends directory contents:
['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__wrapped__']

Geoapify API Key:
8fc9975fd155401ebc2c2ce25ea64708


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.75,75,24,3.13,EC,1731877429
1,1,rensselaer,42.7168,-73.4829,9.69,53,99,1.17,US,1731877430
2,2,waitangi,-43.9535,-176.5597,12.68,68,8,2.24,NZ,1731877431
3,3,iqaluit,63.7506,-68.5145,-3.15,63,75,6.17,CA,1731877433
4,4,aksu,41.1231,80.2644,6.58,44,98,1.47,CN,1731877434


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Import required libraries
import pandas as pd
import folium
from folium.plugins import HeatMap

# Create the base map centered on the mean coordinates
map = folium.Map(
    location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()],
    zoom_start=2,
    tiles='OpenStreetMap'
)

# Add markers for each city
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=6,
        popup=f"City: {row['City']}<br>Max Temp: {row['Max Temp']}°F",
        color='red',
        fill=True,
        fill_opacity=0.7
    ).add_to(map)

# Add title
title_html = '''
             <h3 align="center" style="font-size:16px">
             <b>Max Temperature in World Cities</b>
             </h3>
             '''
map.get_root().html.add_child(folium.Element(title_html))

# Display the map
map

In [7]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    colormap ="magma",
    title="Max Temperature in World Cities",
    frame_width=700,
    frame_height=500,
    grid=False,
    fontsize={"title": 16, "labels": 12, "ticks": 10},
    legend=True,
    color = "City",
    alpha = 0.5
)

# Display the plot
map_plot1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & 
                                   (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 10) & 
                                   (city_data_df["Cloudiness"] < 20)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,39,mulayjah,27.2710,48.4242,23.54,22,0,3.60,SA,1731877477
61,61,dano,11.1500,-3.0667,25.50,17,0,3.40,BF,1731877502
71,71,kirando,-7.4167,30.6000,23.88,79,12,3.85,TZ,1731877514
73,73,enewetak,11.3474,162.3373,28.42,77,3,7.53,MH,1731877516
76,76,adamstown,-25.0660,-130.1015,23.28,77,1,5.70,PN,1731877420


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
39,mulayjah,SA,27.2710,48.4242,
61,dano,BF,11.1500,-3.0667,
71,kirando,TZ,-7.4167,30.6000,
73,enewetak,MH,11.3474,162.3373,
76,adamstown,PN,-25.0660,-130.1015,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        try:
            name_address = response.json()
            # Grab the first hotel from the results and store the name in the hotel_df DataFrame
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # If the request was not successful, set the hotel name as "API request failed".
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



8fc9975fd155401ebc2c2ce25ea64708
Starting hotel search
mulayjah - nearest hotel: No hotel found
dano - nearest hotel: No hotel found
kirando - nearest hotel: Lakeshore Lodge
enewetak - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
utrik - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
arawa - nearest hotel: Bovotel
dawesville - nearest hotel: Baymont Thomasville
ad dabbah - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
rongelap - nearest hotel: No hotel found
el matama - nearest hotel: الكوثر
nguigmi - nearest hotel: Guest PAM
cururupu - nearest hotel: No hotel found
tofol - nearest hotel: Nautilus Resort
liwonde - nearest hotel: Annie's Lodge
wagar - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
veraval - nearest hotel: Shyam
tucuma - nearest hotel: No hotel found
touros - nearest hotel: Pousada Atlântico
filingue - nearest hotel: Complex

,City,Country,Lat,Lng,Hotel Name
39,mulayjah,SA,27.2710,48.4242,No hotel found
61,dano,BF,11.1500,-3.0667,No hotel found
71,kirando,TZ,-7.4167,30.6000,Lakeshore Lodge
73,enewetak,MH,11.3474,162.3373,No hotel found
76,adamstown,PN,-25.0660,-130.1015,No hotel found
80,pisco,PE,-13.7000,-76.2167,La Portada
85,utrik,MH,11.2278,169.8474,No hotel found
87,mahina,PF,-17.5065,-149.4890,Motu Martin
95,arawa,PG,-6.2132,155.5367,Bovotel
116,dawesville,US,30.9224,-84.0091,Baymont Thomasville


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
# Configure the map plot
# Create the base map centered on the mean coordinates

# Configure the map plot
map1 = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    tiles='OSM', 
    color='City', 
    height=500, 
    width=800, 
    hover_cols=["Hotel Name", "Country"]
    )

# Display the map
map1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)